# ETA Evaluation (Colab)
Unified evaluation for ETA on HatefulMemes (train/dev/test) with plots.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [2]:
%pip install -q open_clip_torch transformers huggingface_hub pillow matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00


In [3]:
import os
base_dir = '/content/drive/MyDrive/VLM_project'
image_dir = f'{base_dir}/HatefulMemes/data/img'
train_json = f'{base_dir}/HatefulMemes/data/train_clean.jsonl'
dev_json = f'{base_dir}/HatefulMemes/data/dev_clean.jsonl'
test_json = f'{base_dir}/HatefulMemes/data/test_clean.jsonl'
eta_eval_py = f'{base_dir}/eta_eval.py'
out_train = f'{base_dir}/eta_metrics_train.json'
out_dev = f'{base_dir}/eta_metrics_dev.json'
out_test = f'{base_dir}/eta_metrics_test.json'
plots_dir = f'{base_dir}/plots'
os.makedirs(plots_dir, exist_ok=True)

In [4]:
!python $eta_eval_py --image_dir "$image_dir" --json_path "$train_json" --output_json "$out_train" --k 5 --device cuda
!python $eta_eval_py --image_dir "$image_dir" --json_path "$dev_json" --output_json "$out_dev" --k 5 --device cuda
!python $eta_eval_py --image_dir "$image_dir" --json_path "$test_json" --output_json "$out_test" --k 5 --device cuda

python3: can't open file '/content/drive/MyDrive/VLM_project/eta_eval.py': [Errno 2] No such file or directory
python3: can't open file '/content/drive/MyDrive/VLM_project/eta_eval.py': [Errno 2] No such file or directory
python3: can't open file '/content/drive/MyDrive/VLM_project/eta_eval.py': [Errno 2] No such file or directory


In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
def loadj(p):
    with open(p,'r') as f:
        return json.load(f)
m_train = loadj(out_train)
m_dev = loadj(out_dev)
m_test = loadj(out_test)
def plot_recall(m, title, out_path):
    labels = ['safe_pre','safe_post','unsafe_pre','unsafe_post']
    vals = [m['utility_recall@k_pre_S_V'],m['utility_recall@k_post_S_V'],m['harmful_recall@k_pre_U_V'],m['harmful_recall@k_post_U_V']]
    fig,ax = plt.subplots(figsize=(6,4))
    ax.bar(labels, vals, color=['#4caf50','#81c784','#e53935','#ef9a9a'])
    ax.set_ylim(0,1)
    ax.set_title(title)
    for i,v in enumerate(vals):
        ax.text(i, v+0.02, f'{v:.3f}', ha='center')
    plt.tight_layout()
    plt.savefig(out_path)
    plt.show()
def plot_shift(m, title, out_path):
    labels = ['decline_to_unsafe','increase_to_neutral']
    vals = [m['text_semantic_shift_decline_to_unsafe'],m['text_semantic_shift_increase_to_neutral']]
    fig,ax = plt.subplots(figsize=(6,4))
    ax.bar(labels, vals, color=['#fb8c00','#42a5f5'])
    ax.set_title(title)
    for i,v in enumerate(vals):
        ax.text(i, v+(0.02 if v>=0 else -0.06), f'{v:.3f}', ha='center')
    plt.tight_layout()
    plt.savefig(out_path)
    plt.show()
def plot_clip(m, title, out_path):
    labels = ['clipscore_safe_pre','clipscore_safe_post']
    vals = [m['clipscore_safe_pre'],m['clipscore_safe_post']]
    fig,ax = plt.subplots(figsize=(6,4))
    ax.bar(labels, vals, color=['#8e24aa','#ce93d8'])
    ax.set_title(title)
    for i,v in enumerate(vals):
        ax.text(i, v+0.02, f'{v:.3f}', ha='center')
    plt.tight_layout()
    plt.savefig(out_path)
    plt.show()
plot_recall(m_train,'ETA Recall@5 (Train)',f'{plots_dir}/eta_recall_train.png')
plot_recall(m_dev,'ETA Recall@5 (Dev)',f'{plots_dir}/eta_recall_dev.png')
plot_recall(m_test,'ETA Recall@5 (Test)',f'{plots_dir}/eta_recall_test.png')
plot_shift(m_train,'ETA Semantic Shift (Train)',f'{plots_dir}/eta_shift_train.png')
plot_shift(m_dev,'ETA Semantic Shift (Dev)',f'{plots_dir}/eta_shift_dev.png')
plot_shift(m_test,'ETA Semantic Shift (Test)',f'{plots_dir}/eta_shift_test.png')
plot_clip(m_train,'ETA CLIPScore (Train)',f'{plots_dir}/eta_clip_train.png')
plot_clip(m_dev,'ETA CLIPScore (Dev)',f'{plots_dir}/eta_clip_dev.png')
plot_clip(m_test,'ETA CLIPScore (Test)',f'{plots_dir}/eta_clip_test.png')